In [1]:
using DataStructures

function neighbours(point::CartesianIndex{2}, data::Matrix{Int64})::Vector{CartesianIndex{2}}
    y, x = Tuple(point)
    inds = [                       CartesianIndex((y-1,x)), 
            CartesianIndex((y,x-1)),                      CartesianIndex((y,x+1)),
                                   CartesianIndex((y+1,x))]
    return filter(i->checkbounds(Bool, data, i), inds)
end

function traverse(start, destination, risks, previous)
    p = destination
    shortest_distance = 0
    path = []
    while p != start
        shortest_distance += risks[p]
        p = previous[p]
    end
    return shortest_distance
end 

function expand_risks(filename)
    risks = parse.(Int, permutedims(hcat(collect.(readlines(filename))...)))
    sh = size(risks)
    nrisks = zeros(Int, sh.*5)
    for i=1:5
        for j=1:5
            offset = (i-1) + (j-1)
            nrisks[(i-1)*sh[1]+1:i*sh[1], (j-1)*sh[2]+1:j*sh[2]] = @.((risks + offset - 1) % 9 + 1)
        end
    end
    return nrisks
end

function penalty(loc::CartesianIndex{2}, dest::CartesianIndex{2})::Int64
    # Manhatten distance from destination
    return (abs(dest[1] - loc[1]) +  abs(dest[2] - loc[2]))
end

function a_star(start::CartesianIndex{2}, risks::Array{Int64, 2}, destination::CartesianIndex{2})
    distances = fill(typemax(Int64), size(risks))
    distances[start] = 0
    previous = fill(CartesianIndex((-1, -1)), size(risks))

    visited = Dict{UInt64,CartesianIndex{2}}([])
    frontier = PriorityQueue{CartesianIndex{2}, Int64}([start=>0])

    while length(frontier) > 0
        current = dequeue!(frontier)
        if current == destination
            break
        end
        for n in neighbours(current, risks)
            ndist = distances[current] + risks[n]
            hn = hash(n)
            if (ndist < distances[n]) || !haskey(visited, hn)
                visited[hn] = n
                distances[n] = ndist
                if !haskey(frontier, n)
                    enqueue!(frontier, n=>ndist + penalty(n, destination))
                end
                previous[n] = current
            end
        end
    end
    return previous 
end

a_star (generic function with 1 method)

In [2]:
function solve_part1(filename)
    risks = parse.(Int, permutedims(hcat(collect.(readlines(filename))...)))
    start = CartesianIndex((1,1))
    destination = CartesianIndex(size(risks))

    p = a_star(start, risks, destination)
    shortest_distance = traverse(start, destination, risks, p)
    return shortest_distance
end

println("Part 1 test: $(solve_part1("test.txt"))")
println("Part 1 solution: $(solve_part1("input.txt"))")

Part 1 test: 40
Part 1 solution: 388


In [3]:
function solve_part2(filename)
    risks = expand_risks(filename)
    start = CartesianIndex((1,1))
    destination = CartesianIndex(size(risks))
    @time begin
        p = a_star(start, risks, destination)
    end
    shortest_distance = traverse(start, destination, risks, p)
    return shortest_distance
end

println("Part 2 test: $(solve_part2("test.txt"))")
println("Part 2 solution: $(solve_part2("input.txt"))")

  0.001019 seconds (5.08 k allocations: 991.688 KiB)
Part 2 test: 315
  0.148362 seconds (501.05 k allocations: 103.384 MiB, 10.14% gc time)
Part 2 solution: 2819
